In [1]:
import requests
import pandas as pd
import json
from dateutil import parser

In [2]:
API_KEY = "ab517f7256f412e216a0ce26f3a83405-45fe15cab8bedd85ef35280b7530c467"
BASE_URL = "https://api-fxpractice.oanda.com/v3/"
ACCOUNT_ID = "101-001-23383103-001"

In [3]:
session = requests.Session()

In [4]:
session.headers.update({
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
})

In [5]:
params = dict(
    count = 10,
    granularity = "H1",
    price = "MBA"
)

In [6]:
url = f"{BASE_URL}accounts/{ACCOUNT_ID}/instruments"

In [7]:
response = session.get(url, params=None, data=None, headers=None)

In [8]:
data = response.json()
data

{'instruments': [{'name': 'USD_CZK',
   'type': 'CURRENCY',
   'displayName': 'USD/CZK',
   'pipLocation': -4,
   'displayPrecision': 5,
   'tradeUnitsPrecision': 0,
   'minimumTradeSize': '1',
   'maximumTrailingStopDistance': '1.00000',
   'minimumTrailingStopDistance': '0.00050',
   'maximumPositionSize': '0',
   'maximumOrderUnits': '100000000',
   'marginRate': '0.05',
   'guaranteedStopLossOrderMode': 'DISABLED',
   'tags': [{'type': 'ASSET_CLASS', 'name': 'CURRENCY'}],
   'financing': {'longRate': '-0.067',
    'shortRate': '0.0089',
    'financingDaysOfWeek': [{'dayOfWeek': 'MONDAY', 'daysCharged': 1},
     {'dayOfWeek': 'TUESDAY', 'daysCharged': 1},
     {'dayOfWeek': 'WEDNESDAY', 'daysCharged': 1},
     {'dayOfWeek': 'THURSDAY', 'daysCharged': 1},
     {'dayOfWeek': 'FRIDAY', 'daysCharged': 1},
     {'dayOfWeek': 'SATURDAY', 'daysCharged': 0},
     {'dayOfWeek': 'SUNDAY', 'daysCharged': 0}]}},
  {'name': 'USD_ZAR',
   'type': 'CURRENCY',
   'displayName': 'USD/ZAR',
   'pipLo

In [9]:
instrument_list = data['instruments']

In [10]:
instrument_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [11]:
intrested_keys = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'marginRate']

In [12]:
instruments_dict = {}
for i in instrument_list:
    key = i['name']
    instruments_dict[key] = {k: i[k] for k in intrested_keys}

In [13]:
instruments_dict

{'USD_CZK': {'name': 'USD_CZK',
  'type': 'CURRENCY',
  'displayName': 'USD/CZK',
  'pipLocation': -4,
  'displayPrecision': 5,
  'tradeUnitsPrecision': 0,
  'marginRate': '0.05'},
 'USD_ZAR': {'name': 'USD_ZAR',
  'type': 'CURRENCY',
  'displayName': 'USD/ZAR',
  'pipLocation': -4,
  'displayPrecision': 5,
  'tradeUnitsPrecision': 0,
  'marginRate': '0.07'},
 'EUR_AUD': {'name': 'EUR_AUD',
  'type': 'CURRENCY',
  'displayName': 'EUR/AUD',
  'pipLocation': -4,
  'displayPrecision': 5,
  'tradeUnitsPrecision': 0,
  'marginRate': '0.03'},
 'USD_DKK': {'name': 'USD_DKK',
  'type': 'CURRENCY',
  'displayName': 'USD/DKK',
  'pipLocation': -4,
  'displayPrecision': 5,
  'tradeUnitsPrecision': 0,
  'marginRate': '0.02'},
 'CHF_HKD': {'name': 'CHF_HKD',
  'type': 'CURRENCY',
  'displayName': 'CHF/HKD',
  'pipLocation': -4,
  'displayPrecision': 5,
  'tradeUnitsPrecision': 0,
  'marginRate': '0.1'},
 'NZD_JPY': {'name': 'NZD_JPY',
  'type': 'CURRENCY',
  'displayName': 'NZD/JPY',
  'pipLocation

In [14]:
with open("../data/instruments.json", "w") as f:
    f.write(json.dumps(instruments_dict, indent=2))

In [15]:
def fetch_candles(pair_name, count=10, granularity="H1"):
    url = f"{BASE_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()
    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    return response.status_code, data

def get_candles_df(data):
    if len(data) == 0:
        return pd.DataFrame()
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']
    final_data = []
    for candle in data:
        if candle['complete'] == False: continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']
        final_data.append(new_dict)
        for p in prices:
            for o in ohlc:
                new_dict[(f"{p}_{o}")] = float(candle[p][o])
    df = pd.DataFrame.from_dict(final_data)
    return df

def create_candles_file(pair_name, count=10, granularity="H1"):
    code, data = fetch_candles(pair_name", count=count, granularity="H4")
    if code != 200:
        print("API Error", pair_name, data)
        return
    if len(data) == 0:
        print("No data found")
        return
    candles_df = get_candles_df(data)
    candles_df.to_pickle(f"../data/{pair_name}_{granularity}.pkl")
    print(f"{pair_name} {granularity} {candles_df.shape[0]} candles, {candles_df.time.min()} {candles_df.time.max()}")

SyntaxError: unterminated string literal (detected at line 36) (3166193647.py, line 36)

In [ ]:
code, data = fetch_candles("EUR_USD", count=10, granularity="H4")
candles_df = get_candles_df(data)

In [ ]:
create_candles_file("EUR_USD", count=10, granularity="H4")

In [ ]:
our_currencies = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD', 'CAD', 'AUD']

In [ ]:
for p1 in our_currencies:
    for p2 in our_currencies:
        pr = f"{p1}_{p2}"
        if pr in instruments_dict.keys():
            for granularity in ["H1", "H4"]:
                create_candles_file(pr, count=4000, granularity=granularity)